In [1]:
# Test the updated MCP handling with TAgent
import sys
import os
from dotenv import load_dotenv
load_dotenv()
# sys.path.append('../src')



from t_ai.t_agent import TAgent
from pydantic_ai.mcp import MCPServerStreamableHTTP
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
# Example usage of the updated MCP handling
openai_model = OpenAIModel('gpt-4.1-mini', provider=OpenAIProvider(api_key=os.getenv('openai_api_key')))

# Create MCP server instance (using the new approach)
server1 = MCPServerStreamableHTTP(url=os.getenv('google_task_mcp_server'))  
server2 = MCPServerStreamableHTTP(url=os.getenv('google_calendar_mcp_server'))

# Create agent with MCP server as toolset
agent = TAgent(
    llm=openai_model,
    mcp_servers=[server1,server2],  # Pass as list to TAgent
    system_prompt="You are a helpful assistant with access to MCP tools."
)

print("TAgent with MCP server created successfully!")
print("MCP servers:", len(agent.mcp_servers))


TAgent with MCP server created successfully!
MCP servers: 2


In [2]:
# await server1.list_tools()

In [3]:
# await server1.direct_call_tool('GOOGLETASKS_LIST_TASK_LISTS', {})

In [4]:
# Example of using the async context manager (recommended approach)
async def test_mcp_with_context_manager():
    """
    This demonstrates the recommended way to use MCP servers with context managers
    to properly manage connections and resources.
    """
    async with agent:  # This will connect to all MCP servers
        result = await agent.chat(['How many days between 2000-01-01 and 2025-03-18?'])
        print("Result:", result.output if hasattr(result, 'output') else result)
    # Connections are automatically closed when exiting the context

# Note: This is an async function, so you would run it with:
# await test_mcp_with_context_manager()
print("Async context manager example defined. Use 'await test_mcp_with_context_manager()' to run.")


Async context manager example defined. Use 'await test_mcp_with_context_manager()' to run.


In [5]:
# Alternative: Manual connection management (less efficient)
async def test_mcp_manual_connection(query:str):
    """
    This shows manual connection management. The context manager approach above
    is more efficient as it manages connections around the entire usage context.
    """
    # Manual connection (connections will be managed automatically if not done explicitly)
    result = await agent.chat([query])
    print("Result:", result.output if hasattr(result, 'output') else result)
    
    # Manual disconnection (optional, will be done automatically when needed)
    # await agent.disconnect()

print("Manual connection example defined. Use 'await test_mcp_manual_connection()' to run.")


Manual connection example defined. Use 'await test_mcp_manual_connection()' to run.


In [9]:
result=await test_mcp_manual_connection('list all my tasks')
print(result)

Result: Here are your tasks in the "My Tasks" list:

1. Find information about the dust detection experiment. Due: Jan 28, 2024. Notes: Research and gather details regarding the dust detection experiment. Include objectives, methodology, results, and any relevant publications.
2. Find information about the passive seismic experiment. Due: Dec 31, 2023. Notes: Research and gather data on passive seismic experiments, including their methodology, applications, and relevant case studies.
3. Find information about the laser ranging retroreflector experiment. Due: Dec 31, 2023. Notes: Research the laser ranging retroreflector experiment, its purpose, and results.
4. Find information about the solar wind composition experiment. Due: Oct 27, 2023. Notes: Create a task to find information about the solar wind composition experiment.
5. Find information about the lunar soil mechanics experiment. Due: Dec 31, 2023. Notes: Research and gather data on the lunar soil mechanics experiment.
6. List ex

# Updated MCP Handling in Pydantic AI

## Key Changes Made:

1. **Import Changes**: Added proper MCP imports to `t_agent.py`:
   ```python
   from pydantic_ai.mcp import MCPServerStreamableHTTP, MCPServerSSE, MCPServerStdio
   ```

2. **Agent Creation**: MCP servers are now passed as `toolsets` parameter:
   ```python
   server = MCPServerStreamableHTTP('http://localhost:8000/mcp')  
   agent = Agent('openai:gpt-4o', toolsets=[server])
   ```

3. **Context Manager Support**: Each MCP server instance is a toolset and can be used with async context managers:
   ```python
   async with agent:  
       result = await agent.run('How many days between 2000-01-01 and 2025-03-18?')
   ```

4. **Connection Management**: Updated the TAgent class to properly handle multiple MCP server connections and disconnections.

## Benefits:
- More efficient connection management
- Proper resource cleanup
- Support for multiple MCP servers
- Automatic connection handling when using context managers
